In [1]:
# Import libraries
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import json
import traceback
import numpy as np

fablib = fablib_manager()
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Project ID,0508639f-d515-44c8-b922-cf4cf000b6d9
Token File,/home/fabric/.tokens.json
Bastion Host,bastion.fabric-testbed.net
Bastion Username,bschlue_0000009427
Bastion Private Key File,/home/fabric/work/fabric_config/bastion-key
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Log File,/tmp/fablib/fablib.log
Log Level,INFO


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Project ID,0508639f-d515-44c8-b922-cf4cf000b6d9
Token File,/home/fabric/.tokens.json
Bastion Host,bastion.fabric-testbed.net
Bastion Username,bschlue_0000009427
Bastion Private Key File,/home/fabric/work/fabric_config/bastion-key
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Log File,/tmp/fablib/fablib.log
Log Level,INFO


In [2]:
# Setup constants
cores = 1
ram = 2
disk = 10
nBots = 4
nNodes = 3 + nBots

In [8]:
# Define and Submit Slice
slice_name = "ECE6400-Final-DDoS-Sim"
site = "EDUKY"
print(site)

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

network_name="network"

botNodes = [None] * nBots
network_interfaces = [None] * nNodes

slice = fablib.new_slice(name=slice_name)

# Create all of the nodes for the network
userNode = slice.add_node(name="User", site=site)
userNode.set_capacities(cores=cores, ram=ram, disk=disk)
userNode.set_image(image)    
user_ap = userNode.add_component(model=nicmodel, name="user-nic").get_interfaces()[0]
network_interfaces[nNodes - 3] = user_ap

targetNode = slice.add_node(name="Target", site=site)
targetNode.set_capacities(cores=cores, ram=ram, disk=disk)
targetNode.set_image(image)    
target_ap = targetNode.add_component(model=nicmodel, name="target-nic").get_interfaces()[0] 
network_interfaces[nNodes - 2] = target_ap

for i in range(0, nBots):
    botNodes[i] = slice.add_node(name="Bot_" + str(i), site=site)
    botNodes[i].set_capacities(cores=cores, ram=ram, disk=disk)
    botNodes[i].set_image(image)    
    network_interfaces[i] = botNodes[i].add_component(model=nicmodel, name="bot-nic-" + str(i)).get_interfaces()[0] 

controllerNode = slice.add_node(name="Controller", site=site)
controllerNode.set_capacities(cores=cores, ram=ram, disk=disk)
controllerNode.set_image(image)    
controller_ap = controllerNode.add_component(model=nicmodel, name="controller-nic").get_interfaces()[0]
network_interfaces[nNodes - 1] = controller_ap

# Network
l2_network = slice.add_l2network(name=network_name, interfaces=network_interfaces)


#Submit Slice Request
slice.submit()



Retry: 10, Time: 280 sec


ID,143b9cb6-adff-4b07-a107-492baeb4f7fd
Name,ECE6400-Final-DDoS-Sim
Lease Expiration (UTC),2025-04-01 19:09:00 +0000
Lease Start (UTC),2025-03-31 19:09:00 +0000
Project ID,0508639f-d515-44c8-b922-cf4cf000b6d9
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
dc06fea6-33ff-49e6-827c-34fe30c8d69c,Bot_0,1,2,10,default_ubuntu_20,qcow2,eduky-w11.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe68:e744,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe68:e744,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
93c9685c-295f-4b41-bbc6-c6d18a00eabd,Bot_1,1,2,10,default_ubuntu_20,qcow2,eduky-w8.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe6d:45cf,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe6d:45cf,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c400eb56-0bf1-46b9-b625-4fdfafd4f854,Bot_2,1,2,10,default_ubuntu_20,qcow2,eduky-w3.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fed5:35c0,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fed5:35c0,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
60bd3329-9fa9-4ddc-95da-1ec9d5f67381,Bot_3,1,2,10,default_ubuntu_20,qcow2,eduky-w2.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fefc:51e4,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fefc:51e4,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
55ad28d3-5909-43a9-b719-5594a0408a2e,Controller,1,2,10,default_ubuntu_20,qcow2,eduky-w10.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe19:5ed3,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe19:5ed3,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
5ad17fb6-26a0-403e-aec8-b23ea1c0e04a,Target,1,2,10,default_ubuntu_20,qcow2,eduky-w5.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe28:f9d8,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe28:f9d8,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
b80a195c-cea2-4ee9-9082-e17acb9c8477,User,1,2,10,default_ubuntu_20,qcow2,eduky-w9.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe4a:4a90,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe4a:4a90,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
e8fdabca-541b-44e4-94f3-b09317956563,network,L2,L2Bridge,EDUKY,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
User-user-nic-p1,p1,User,network,100,config,,02:F6:E1:4D:95:17,enp7s0,enp7s0,fe80::f6:e1ff:fe4d:9517,1,HundredGigE0/0/0/9
Target-target-nic-p1,p1,Target,network,100,config,,06:6F:C2:5E:81:3D,enp7s0,enp7s0,fe80::46f:c2ff:fe5e:813d,1,HundredGigE0/0/0/5
Bot_0-bot-nic-0-p1,p1,Bot_0,network,100,config,,02:80:63:DF:71:DC,enp7s0,enp7s0,fe80::80:63ff:fedf:71dc,1,HundredGigE0/0/0/11
Bot_1-bot-nic-1-p1,p1,Bot_1,network,100,config,,0A:F8:83:CD:8A:6D,enp7s0,enp7s0,fe80::8f8:83ff:fecd:8a6d,1,HundredGigE0/0/0/26
Bot_2-bot-nic-2-p1,p1,Bot_2,network,100,config,,12:11:B6:B9:4C:F8,enp7s0,enp7s0,fe80::1011:b6ff:feb9:4cf8,1,HundredGigE0/0/0/21
Bot_3-bot-nic-3-p1,p1,Bot_3,network,100,config,,02:F2:7C:37:22:3E,enp7s0,enp7s0,fe80::f2:7cff:fe37:223e,1,HundredGigE0/0/0/20
Controller-controller-nic-p1,p1,Controller,network,100,config,,06:22:4A:5B:D8:CC,enp7s0,enp7s0,fe80::422:4aff:fe5b:d8cc,1,HundredGigE0/0/0/28



Time to print interfaces 298 seconds


'143b9cb6-adff-4b07-a107-492baeb4f7fd'

In [9]:
cidr = "16"

ip_user = "10.0.0.1"
ip_target = "10.0.0.2"
ip_controller = "10.0.0.254"

iface_bots = [None] * nBots

# get slice info 

bots = [None] * nBots

user = slice.get_node(name="User")
target = slice.get_node(name="Target")

for i in range(0, nBots):
    bots[i] = slice.get_node(name=f"Bot_{str(i)}")

controller = slice.get_node(name="Controller")

# remove any existing IP's from the interface to prevent duplicates from being added

user.execute("sudo ip addr flush enp7s0")
target.execute("sudo ip addr flush enp7s0")
controller.execute("sudo ip addr flush enp7s0")

for i in range(0, nBots):
    bots[i].execute("sudo ip addr flush enp7s0")

# set interface IP's
iface_user = user.get_interface(network_name=network_name)
iface_user.set_ip(ip=ip_user, cidr=cidr)

iface_target = target.get_interface(network_name=network_name) #target to user
iface_target.set_ip(ip=ip_target, cidr=cidr)

iface_controller = controller.get_interface(network_name=network_name)
iface_controller.set_ip(ip=ip_controller, cidr=cidr)

for i in range(0, nBots): 
    iface_bots[i] = bots[i].get_interface(network_name=network_name)
    iface_bots[i].set_ip(ip=f"10.0.1.{str(i)}", cidr=cidr)
    



In [10]:
# IP space where bot cannot access user (realistic)

# 10.0.0.x represents the network the client and the target share
# 10.0.1.x represents the network the client and bot share
# 192.168.0.x is the network all the bots and the controller are on

ip_user = "10.0.0.1"
ip_target_u = "10.0.0.2"
ip_target_b = "10.0.1.1"
ip_bot_t = "10.0.1.2"
ip_bot_c = "192.168.0.1"
ip_controller = "192.168.0.254"

bots = [None] * nBots

user = slice.get_node(name="User")
target = slice.get_node(name="Target")

for i in range(0, nBots):
    bots[i] = slice.get_node(name=f"Bot_{str(i)}")

controller = slice.get_node(name="Controller")

# remove any existing IP's from the interface to prevent duplicates from being added
user.execute("sudo ip addr flush enp7s0")

for i in range(0, nBots):
    target.execute("sudo ip addr flush enp7s0")
    bots[i].execute("sudo ip addr flush enp7s0")
    target.execute("sudo ip addr flush enp8s0")
    bots[i].execute("sudo ip addr flush enp8s0")
    
controller.execute("sudo ip addr flush enp7s0")

# set interface IP's
iface_user = user.get_interface(network_name="tu_bridge")
iface_user.set_ip(ip=ip_user, cidr="24")

iface_target_u = target.get_interface(network_name="tu_bridge") #target to user
iface_target_u.set_ip(ip=ip_target_u, cidr="24")
iface_target_b = target.get_interface(network_name="bt_bridge") #target to bot
iface_target_b.set_ip(ip=ip_target_b, cidr="24")

iface_bot_t = bot.get_interface(network_name="bt_bridge") #bot to target
iface_bot_t.set_ip(ip=ip_bot_t, cidr="24")
iface_bot_c = bot.get_interface(network_name="cb_bridge") #bot to controller
iface_bot_c.set_ip(ip=ip_bot_c, cidr="24")

iface_controller = controller.get_interface(network_name="cb_bridge")
iface_controller.set_ip(ip=ip_controller, cidr="24")


 Device "enp8s0" does not exist.
  Device "enp8s0" does not exist.
  Device "enp8s0" does not exist.
  Device "enp8s0" does not exist.
  Device "enp8s0" does not exist.
  Device "enp8s0" does not exist.
  Device "enp8s0" does not exist.
  Device "enp8s0" does not exist.
 

Exception: Interface not found: None

In [ ]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = True
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")